<div align="right"><i>William Quiroga y Matías Torres Esteban<br>Agosto, 2025</i></div>

# Aumentación de Datos con Grandes Modelos de Lenguaje

En el año 2020, nuestro grupo de investigación participó en el Proyecto de Transferencia de Tecnología: “BillMobile: un prototipo de chatbot basado en IA para mejorar la calidad de experiencia del cliente”. Como resultado, se obtuvo una colección de preguntas relacionadas con el Régimen de Monotributo de la Administración Federal de Ingresos Públicos (AFIP). Durante el proyecto, se aplicaron diversos enfoques de aprendizaje automático, incluyendo modelos basados en Transformers.

A pesar de haberse desarrollado un estudio integral, uno de los principales desafíos fue la **cantidad limitada de ejemplos por clase en la colección de preguntas**, lo que restringió el desempeño de los modelos entrenados.

Proponemos diseñar una estrategia de aumentación de datos utilizando modelos generativos actuales para ampliar la colección original de preguntas. El objetivo es generar nuevas muestras  que enriquezcan el conjunto de datos en cada una de las clases definidas durante el proceso de etiquetado.

*  Formular prompts optimizados para generar nuevas preguntas sintéticas, cuidando que estén alineadas semánticamente con cada categoría.
*   Utilizar los resultados obtenidos en la Notebook anterior para ajustar y refinar dichos prompts, priorizando la calidad y relevancia de las muestras generadas.

### Taxonomía de Preguntas

Esta taxonomía clasifica los tipos de preguntas para facilitar la organización y el análisis de la información. Cada categoría describe el tipo de respuesta esperada y ofrece ejemplos claros.*

---
1. **DEFINICIÓN**

**Descripción:** Preguntas que buscan respuestas descriptivas y conceptuales. Apuntan a la definición de un concepto.

**Ejemplos:**

¿Qué es...?

¿Qué se entiende por...?

¿Cuál es la diferencia entre...?

---
2. **REFERENCIA**

**Descripción:** Preguntas que requieren referencias simples, excluyendo ubicación geográfica, expresiones temporales y entidades específicas.

**Ejemplos:**

¿Cuál es el número de teléfono / página web /...?

¿Dónde consultar...? (en qué sitio web)

¿Cuáles son las categorías vigentes? (ref. a página web / documentación)

¿Qué comprobante emite un monotributista? (se indica tipo de comprobante)

¿Qué letra deben contener los remitos? (se indica tipo de letra)

---
3. **UBICACIÓN**

**Descripción:** Preguntas que solicitan una referencia a una ubicación geográfica.

**Ejemplos:**

¿Dónde debo solicitar el CUIT? (en qué dependencia de AFIP - ubicación geográfica)

---
4. **TEMPORAL**

**Descripción:** Preguntas que hacen referencia a un tiempo, intervalo o periodo particular.

**Ejemplos:**

¿A partir de cuándo...?

¿Cuándo se debe realizar el primer pago...?

---
5. **ENTIDAD**

**Descripción:** Preguntas que buscan una referencia a una entidad (persona, institución, organismo, etc.).

**Ejemplos:**

¿Cuáles son las entidades bancarias habilitadas para...?

---
6. **RAZÓN**

**Descripción:** Preguntas que indagan sobre razones, condiciones, casos o excepciones que conforman una causa. Apuntan a entender una causa que luego deriva en un efecto (relación de causalidad).

**Ejemplos:**

¿En qué casos los monotributistas no se encuentran obligados a...? (casos de excepción)

¿Cuáles son los requisitos para inscribirse a...? (condiciones)

¿Quiénes pueden ser monotributistas? (condiciones para acceder al régimen)

---
7. **PROCEDIMIENTO**

**Descripción:** Preguntas que esperan respuestas prescriptivas o instructivas, incluyendo una sucesión de pasos explícitos con dependencia entre sí.

**Ejemplos:**

¿Cómo es el procedimiento para darme de baja?

¿Cuáles son los pasos para recategorizarme?

---
8. **MANERA**

**Descripción:** Preguntas sobre formas, maneras, opciones o parámetros para realizar una determinada acción. Involucran respuestas descriptivas que apuntan a la explicación general de cómo realizar una actividad.

**Ejemplos:**

¿Qué formas de pago tengo para...? (opciones de pago)

¿Cuáles son los parámetros considerados para recategorizar? (parámetros para efectuar el trámite)

De realizar más de una actividad como monotributista, ¿cómo se determina cuál es la principal? (forma para determinar una actividad principal)

¿Cuál es el tratamiento aplicable a los condominios? (formas de adhesión)

---
9. **CANTIDAD**

**Descripción:** Preguntas que buscan la cantidad o el monto de un determinado concepto.

**Ejemplos:**

¿Cuál es el precio máximo unitario de venta?

---
10. **SI/NO**

**Descripción:** Preguntas que requieren una respuesta booleana (SI o NO).

**Ejemplos:**

¿Se puede...?

¿Estoy obligado a...?

### El Proceso

En el siguiente trozo de código importamos las funciones
necesarias para realizar el ejercicio y establecemos las configuraciones generales. Ustedes deberían modificar el nombre del modelo de lenguaje si hiciera falta.

In [ ]:
from aumentacion import *

labels = [
    'cantidad',
    'definicion',
    'entidad',
    'manera',
    'procedimiento',
    'razon',
    'referencia',
    'si_no',
    'temporal',
    'ubicacion'
]

dataset_path = './dataset.csv'
template_path = './template.txt'
augmented_set_path = './augmented_set.csv'
classifier_path = './classifier'

llm_data = {
    'url': "http://localhost:1234/v1/chat/completions",
    'llm_name': 'mistralai/mistral-7b-instruct-v0.3'
}

Aquí cargamos el lote de datos original, la plantilla del prompt e invocamos al modelo de lenguaje para que aumente nuestro conjunto de datos.

In [ ]:
dataset = load_dataset(dataset_path, labels)
template = get_file_content(template_path)

llm_answers = ask_llm_to_augment(labels, dataset, template, llm_data)
augmented_set = parse_llm_answers(llm_answers)
save_as_csv(augmented_set, augmented_set_path)

Cargamos el lote de datos aumentado y evaluamos que tan buena fue la aumentación.

In [ ]:
augmented_set = load_augmented_set(augmented_set_path)

predictions = ask_classifier_to_predict(classifier_path, labels, augmented_set['question'])
eval_augmented_set(augmented_set['label'], predictions)